In [1]:
# 文件：models/transformer_model.py

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kendalltau
from tqdm import tqdm
import pandas as pd
import numpy as np

# 参数设置
BATCH_SIZE = 64
EPOCHS = 50
LR = 1e-4
SEQ_LEN = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class TransformerModel(nn.Module):
    def __init__(self, feature_size, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.input_fc = nn.Linear(feature_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.output_fc(x).squeeze()

def reshape_series(X, y, seq_len):
    n_samples = len(X) // seq_len
    X_new = X[:n_samples * seq_len].reshape(n_samples, seq_len, -1)
    y_new = y[seq_len - 1::seq_len][:n_samples]
    return X_new, y_new

def create_sliding_windows(X, seq_len):
    n_samples = len(X) - seq_len + 1
    X_new = np.zeros((n_samples, seq_len, X.shape[1]))
    for i in range(n_samples):
        X_new[i] = X[i:i+seq_len]
    return X_new

X_train = pd.read_parquet("../kaggle/input/X_train.parquet").fillna(0).values
y_train = pd.read_parquet("../kaggle/input/y_train.parquet").squeeze().values
X_train, y_train = reshape_series(X_train, y_train, SEQ_LEN)

X_test_raw = pd.read_parquet("../kaggle/input/X_test.parquet").fillna(0).values
expected_test_rows = len(X_test_raw)
padding = np.zeros((SEQ_LEN-1, X_test_raw.shape[1]))
X_test_padded = np.vstack([padding, X_test_raw])
X_test = create_sliding_windows(X_test_padded, SEQ_LEN)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
train_loader = DataLoader(TimeSeriesDataset(X_tr, y_tr), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=BATCH_SIZE)

model = TransformerModel(feature_size=X_train.shape[2]).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.MSELoss()
best_r2 = -np.inf

for epoch in range(EPOCHS):
    model.train()
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for X_batch, y_batch in loop:
        X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())

    model.eval()
    preds_list, y_list = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(DEVICE)
            preds = model(X_batch).cpu().numpy()
            preds_list.extend(preds)
            y_list.extend(y_batch.numpy())

    r2 = r2_score(y_list, preds_list)
    pearson = pearsonr(y_list, preds_list)[0]
    spearman = spearmanr(y_list, preds_list)[0]
    kendall = kendalltau(y_list, preds_list)[0]
    print(f"R2: {r2:.4f}, Pearson: {pearson:.4f}, Spearman: {spearman:.4f}, Kendall: {kendall:.4f}")

    if r2 > best_r2:
        best_r2 = r2
        torch.save(model.state_dict(), "../models/transformer_best.pth")


Using device: cuda


Epoch 1/50:   0%|          | 0/658 [00:00<?, ?it/s]c:\Users\Jacky\anaconda3\envs\torch\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 1/50: 100%|██████████| 658/658 [00:20<00:00, 32.11it/s, loss=1.14] 


R2: 0.0170, Pearson: 0.1545, Spearman: 0.0937, Kendall: 0.0635


Epoch 2/50: 100%|██████████| 658/658 [00:19<00:00, 34.51it/s, loss=1.69] 


R2: 0.0552, Pearson: 0.2390, Spearman: 0.1319, Kendall: 0.0893


...

R2: 0.6137, Pearson: 0.7879, Spearman: 0.6932, Kendall: 0.5202


Epoch 49/50: 100%|██████████| 658/658 [00:04<00:00, 142.05it/s, loss=0.205]


R2: 0.5817, Pearson: 0.7832, Spearman: 0.6814, Kendall: 0.5096


Epoch 50/50: 100%|██████████| 658/658 [00:04<00:00, 134.58it/s, loss=0.176] 


R2: 0.6031, Pearson: 0.7852, Spearman: 0.6865, Kendall: 0.5130


In [2]:

model.load_state_dict(torch.load("../models/transformer_best.pth", map_location=DEVICE))
model.eval()
test_loader = DataLoader(TimeSeriesDataset(X_test, np.zeros(len(X_test))), batch_size=BATCH_SIZE, shuffle=False)
test_preds = []
with torch.no_grad():
    for X_batch, _ in tqdm(test_loader, desc="Generating predictions"):
        X_batch = X_batch.to(DEVICE)
        preds = model(X_batch).cpu().numpy()
        test_preds.extend(preds)

assert len(test_preds) == expected_test_rows, f"预测数量({len(test_preds)})与测试集行数({expected_test_rows})不匹配"
submission = pd.DataFrame({"ID": range(1, len(test_preds) + 1), "prediction": test_preds})
submission.to_csv("../result/sub_transformer.csv", index=False)
print("Transformer模型提交文件已保存")

Generating predictions: 100%|██████████| 8409/8409 [00:12<00:00, 691.62it/s]


Transformer模型提交文件已保存
